In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests 
from bs4 import BeautifulSoup
import time
import math
import pandas as pd
from google.colab import drive
drive.mount('drive')

     |████████████████████████████████| 911kB 9.1MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,688 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:12 https://deve

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


def timeRow():
  localtime = time.asctime( time.localtime(time.time()) )
  timeList = []
  timeList.append("updated")
  
  while localtime.find(" ") != -1:
    if localtime[0:localtime.find(" ")] != '':
      timeList.append(localtime[0:localtime.find(" ")])
    localtime = localtime[localtime.find(" ")+1:]

  last = timeList[-1]
  timeList[-1] = localtime
  localtime = last

  while localtime.find(":") != -1:
    timeList.append(localtime[0:localtime.find(":")+1])
    localtime = localtime[localtime.find(":")+1:]
  
  timeList.append(localtime)
  timeList.append('GMT+3:30')
  timeListLen = len(timeList)

  for i in range(9-timeListLen):
    timeList.append("")
  dicTime = {'State Name':timeList[0],'Percentage of Counted Votes':timeList[1],'Counted Votes for Biden':timeList[2],'Counted Votes for Trump':timeList[3],'Uncounted Votes':timeList[4],'Vote Biden Needed to Win':timeList[-4],'Percentage of Uncounted Vote Biden Needed to Win':timeList[-3],'Vote Trump Needed to Win':timeList[-2],'Percentage of Uncounted Vote Trump Needed to Win':timeList[-1]}
  return(dicTime)

firstData = True
while True:
  driver.get('https://www.google.com/search?q=us+election+results&oq=us+election+results&aqs=chrome.0.69i59j0i131i433l6j69i60.8640j0j9&sourceid=chrome&ie=UTF-8')
  timeout = 3
  try:
      element_present = EC.presence_of_element_located((By.ID, '_83GlX73aK5_G4-EPqtC9uAQ95'))
      WebDriverWait(driver, timeout).until(element_present)
  except TimeoutException:
      print("Timed out waiting for page to load")
  finally:
      print("Page loaded")
      userid_element = driver.find_elements_by_xpath('//div[@Style="width:172px;text-align:center"]')[0]
      
      userid_element = driver.find_elements_by_xpath('//span[@class="PLaf5"]')
      userid_element1 = driver.find_elements_by_xpath('//span[@class="E2WDKf"]')


      data = []
      for i in userid_element:
        if i.text != "":
          data.append(i.text)
      for i in userid_element1:
        if i.text != "":
          data.append(int(i.text.replace('% reporting','')))
      for i in range(int(len(data)/2)):
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[1].text.replace(',','')))
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[3].text.replace(',','')))



      print(data)
      rawData = []
      for i in range(int(len(data)/4)):
        d = []
        for j in range(2):
          d.append(data[i+j*6])
        d.append(data[12+i*2])
        d.append(data[13+i*2])
        rawData.append(d)
      

      
      info =[]
      data = rawData
      for i in range(len(data)):
        raw = data[i]
        d =[]
        d.append(raw[0])
        diff = raw[2] - raw[3]
        percent = raw[1]/100.0
        uncountedVotes = math.floor((raw[2]+raw[3])*(1-percent)/percent)
        if diff < 0:
          voteNeeded = math.ceil(((uncountedVotes + diff)/2) -diff)
        else:
          voteNeeded = math.ceil(((uncountedVotes - diff)/2))

        percentNeeded = voteNeeded/(uncountedVotes*1.0)
        d.append(str(raw[1])+"%")
        d.append(raw[2])
        d.append(raw[3])
        if percentNeeded < 1 and voteNeeded > 0:
          d.append(uncountedVotes)
          d.append(voteNeeded)
          d.append(str(math.ceil(percentNeeded*1000)/10.0)+"%")
          d.append(uncountedVotes+1-voteNeeded)
          d.append(str(math.ceil((((uncountedVotes+1-voteNeeded)/uncountedVotes)*1000))/10.0)+"%")
          info.append(d)
          
        elif voteNeeded <= 0:
          d.append(uncountedVotes)
          d.append('Already Won')
          d.append('Already Won')
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          info.append(d)
          
        elif percentNeeded >= 1:
          d.append(uncountedVotes)
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          d.append('Already Won')
          d.append('Already Won')
          info.append(d)
          
      if firstData == True:
        print(info)
        df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Trump Needed to Win'])
        dicTime = timeRow()
        df = df.append(dicTime, ignore_index=True)
        mainPath =r'drive/My Drive/New Folder'
        csvPath =mainPath+"/"+"USElectionSwingState.csv"
        df.to_csv(csvPath)
        
        csvPath =mainPath+"/"+"USElectionSwingState"+str(len(info))+".csv"
        df.to_csv(csvPath)

        oldInfo = info
        firstData = False
      else:
        flagDataChanged = False
        for i in range(len(info)):
          new = info[i]
          for j in range(len(oldInfo)):
            old = oldInfo[j]
            if new[0] == old[0]:
              for k in range(len(new)-1):
                if new[k+1] != old[k+1]:
                  flagDataChanged = True
                  print('changed '+str(old[k+1])+' to '+str(new[k+1]))
                  break
              break
          if flagDataChanged == True:
            print(info)
            df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Trump Needed to Win'])
            dicTime = timeRow()
            df = df.append(dicTime, ignore_index=True)
            mainPath =r'drive/My Drive/New Folder'
            csvPath =mainPath+"/"+"USElectionSwingState.csv"
            df.to_csv(csvPath)

            csvPath =mainPath+"/"+"USElectionSwingState"+str(len(info))+".csv"
            df.to_csv(csvPath)

            oldInfo = info
            break


      time.sleep(120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Timed out waiting for page to load
Page loaded
['Georgia', 'North Carolina', 'Arizona', 'Florida', 'Iowa', 'Michigan', 99, 99, 98, 99, 99, 99, 2468003, 2456590, 2659706, 2734525, 1645586, 1630700, 5284453, 5658847, 757905, 896411, 2790648, 2644525]
[['Georgia', '99%', 2468003, 2456590, 49743, 19165, '38.6%', 30579, '61.5%'], ['North Carolina', '99%', 2659706, 2734525, 54487, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won'], ['Arizona', '98%', 1645586, 1630700, 66862, 25988, '38.9%', 40875, '61.2%'], ['Florida', '99%', 5284453, 5658847, 110538, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won'], ['Iowa', '99%', 757905, 896411, 16710, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won'], ['Michigan', '99%', 2790648, 2644525, 54900, 'Already Won', 'Already Won', 'Impossible to Win in This World', 'Impossible to Win in This World']]
Timed out wa